In [1]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

/home/poli/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Arbol que paso David:
```
(0 (A a) (1 (2 (3 (B b) (4 (C c) (5 (6 (D d) (E e)) (F f)))) (G g)) (H h))) <br>
```


## Testing from file

In [2]:
# tree_string = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
# tree_string = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
# tree_string = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
# tree_string = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
# tree_string = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"

# this one has problem with the * in the end
tree_string = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
Tree.fromstring(tree_string).pretty_print()
tetraencoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False, binary_marker="~")
gold_tree = C_Tree.from_string(tree_string)
lin_tree = tetraencoder.encode(gold_tree)
print(lin_tree)
print("====================================")
decoded_tree = tetraencoder.decode(lin_tree)
print("[G]", gold_tree)
print("[D]", decoded_tree)
Tree.fromstring(str(decoded_tree)).pretty_print()
print(str(decoded_tree) == str(gold_tree))

                 NP             
  _______________|____________   
 |          VP                | 
 |      ____|____             |  
 NP    |         NP           X 
 |     |     ____|_______     |  
 NN   VBN  NNP   CD  ,   CD   NN
 |     |    |    |   |   |    |  
Year ended Dec.  31  ,  1988  \*

-BOS-	-BOS-	-BOS-
Year	NN	rR_NP_NP
ended	VBN	rR_VP
Dec.	NNP	rL_NP
31	CD	rL_NP~
,	,	rL_NP~
1988	CD	lL_NP~
\*	NN	l_NONE_X
-EOS-	-EOS-	-EOS-

[G] (NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))
[D] (NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))
                 NP             
  _______________|____________   
 |          VP                | 
 |      ____|____             |  
 NP    |         NP           X 
 |     |     ____|_______     |  
 NN   VBN  NNP   CD  ,   CD   NN
 |     |    |    |   |   |    |  
Year ended Dec.  31  ,  1988  \*

True


In [3]:
dec_tree = tetraencoder.decode(lin_tree)
print(dec_tree)

(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))
